In [1]:
from openmm import app as app
from openmm import unit as unit
import nglview as nv
from pathlib import Path
import numpy as np
import mdtraj as md
root = Path("/home/gridsan/ddavid/MD/")

In [ ]:
def process(input,output):
    pdb = app.PDBFile(input)

    modeller = app.Modeller(pdb.topology, pdb.positions)

    forcefield = app.ForceField('amber14/protein.ff14SB.xml','amber14/tip3p.xml')

    # Add hydrogen atoms at pH 7.4 (default pH is 7.0)
    modeller.addHydrogens(forcefield, pH=7.4)

    modeller.addSolvent(forcefield, model='tip3p', ionicStrength=0.15*unit.molar, neutralize=True, padding=1.0*unit.nanometers)

    with open(output, 'w') as file:
        app.PDBFile.writeFile(modeller.topology, modeller.positions, file)
    print(f"Modified system saved to: {output}")

In [ ]:
def rotate_positions(positions, angle, axis='z'):
    """Rotate positions around a specified axis by a given angle."""
    if axis == 'z':
        rotation_matrix = np.array([[np.cos(angle), -np.sin(angle), 0],
                                    [np.sin(angle), np.cos(angle), 0],
                                    [0, 0, 1]])
    elif axis == 'y':
        rotation_matrix = np.array([[np.cos(angle), 0, np.sin(angle)],
                                    [0, 1, 0],
                                    [-np.sin(angle), 0, np.cos(angle)]])
    elif axis == 'x':
        rotation_matrix = np.array([[1, 0, 0],
                                    [0, np.cos(angle), -np.sin(angle)],
                                    [0, np.sin(angle), np.cos(angle)]])
    else:
        raise ValueError("Axis must be 'x', 'y', or 'z'")

    rotated_positions = np.dot(positions, rotation_matrix)
    return rotated_positions

# Single HRT

In [ ]:
# Load PDB
process((root/"PDBs/HRT.pdb").as_posix(),(root / "PDBs/HRT_Single.pdb").as_posix())

In [ ]:
v = nv.show_file("/home/gridsan/ddavid/MD/PDBs/HRT_Single.pdb")
v.clear()
v.add_cartoon()
v.add_licorice("HOH")
v

# Double HRT

In [ ]:
# Load PDB
pdb = app.PDBFile((root/"PDBs/HRT.pdb").as_posix())

modeller = app.Modeller(pdb.topology, pdb.positions)

forcefield = app.ForceField('amber14/protein.ff14SB.xml','amber14/tip3p.xml')

pdb2_positions_np = np.array([list(atom_pos.value_in_unit(unit.nanometers)) for atom_pos in pdb.positions])
rotated_positions_np = rotate_positions(pdb2_positions_np, np.pi, axis='x')
rotated_positions_np = rotate_positions(rotated_positions_np, np.pi/4, axis='y')
translated_positions_np = rotated_positions_np + np.array([2, 2, -6])
modeller.add(pdb.topology, translated_positions_np * unit.nanometers)

# Add hydrogen atoms at pH 7.4 (default pH is 7.0)
modeller.addHydrogens(forcefield, pH=7.4)

modeller.addSolvent(forcefield, model='tip3p', ionicStrength=0.15*unit.molar, neutralize=True, padding=1.0*unit.nanometers)


output_pdb_path = (root / "PDBs/HRT_Double.pdb").as_posix()
with open(output_pdb_path, 'w') as file:
    app.PDBFile.writeFile(modeller.topology, modeller.positions, file)
print(f"Modified system saved to: {output_pdb_path}")

In [ ]:
v = nv.show_file("/home/gridsan/ddavid/MD/PDBs/HRT_Double.pdb")
v.clear()
v.add_spacefill()
# v.add_point("HOH")
v

# Single PhaC


In [ ]:
process((root/"PDBs/PhaC.pdb").as_posix(),(root / "PDBs/Phac_Single.pdb").as_posix())

In [ ]:
v = nv.show_file((root / "PDBs/Phac_Single.pdb").as_posix())
v.clear()
v.add_cartoon()
v

# Two PhaC


In [7]:
# Load PDB
pdb = app.PDBFile((root/"PDBs/PhaC.pdb").as_posix())

modeller = app.Modeller(pdb.topology, pdb.positions)

forcefield = app.ForceField('amber14/protein.ff14SB.xml','amber14/tip3p.xml')

pdb2_positions_np = np.array([list(atom_pos.value_in_unit(unit.nanometers)) for atom_pos in pdb.positions])
rotated_positions_np = rotate_positions(pdb2_positions_np, np.pi, axis='x')
rotated_positions_np = rotate_positions(rotated_positions_np, np.pi/4, axis='y')
translated_positions_np = rotated_positions_np + np.array([2, 2, -8])
modeller.add(pdb.topology, translated_positions_np * unit.nanometers)

# Add hydrogen atoms at pH 7.4 (default pH is 7.0)
# modeller.addHydrogens(forcefield, pH=7.4)

# modeller.addSolvent(forcefield, model='tip3p', ionicStrength=0.15*unit.molar, neutralize=True, padding=1.0*unit.nanometers)


output_pdb_path = (root / "PDBs/Phac_Double.pdb").as_posix()
with open(output_pdb_path, 'w') as file:
    app.PDBFile.writeFile(modeller.topology, modeller.positions, file)
print(f"Modified system saved to: {output_pdb_path}")

Modified system saved to: /home/gridsan/ddavid/MD/PDBs/Phac_Double.pdb


In [8]:
v = nv.show_file((root / "PDBs/Phac_Double.pdb").as_posix())
v.clear()
v.add_cartoon()
v

NGLWidget()